In [1]:
import torch

# Data preparation

## Load raw text

In [2]:
with open('../data/shakespeare.txt', 'r') as f:
    text = f.read()

In [4]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


## Tokenization

In [8]:
tokens = sorted(list(set(text)))
''.join(tokens)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [9]:
stoi = { ch: i for i, ch in enumerate(tokens) }
itos = { i: ch for i, ch in enumerate(tokens) }

In [20]:
def encode(text):
    return torch.tensor([stoi[ch] for ch in text], dtype=torch.long)

def decode(tensor):
    return ''.join([itos[i.item()] for i in tensor])

In [21]:
encode('testi')

tensor([58, 43, 57, 58, 47])

In [22]:
decode(encode('testi'))

'testi'

In [24]:
data = encode(text)
data[:10]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])

In [25]:
decode(data[:10])

'First Citi'

In [27]:
split = int(0.8 * len(data))
train = data[:split]
val = data[split:]

print(train.shape, val.shape)

torch.Size([892315]) torch.Size([223079])


# Model